In [10]:
#miss forest doesn't work for current version of sklearn
#if necessary, downgrade sklearn version
#https://github.com/scikit-learn/scikit-learn/discussions/25745

#pip uninstall scikit-learn -y
#pip install scikit-learn==1.1.2

In [14]:
import sspa
import scipy
print(sspa.__version__)

0.2.4


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#ignores future warnings for miss forest

In [11]:
import pandas as pd
import openpyxl   #to read in a sheet of the excel file

#imputation libraries  (import sklearn before importing sys and missingpy otherwise an error will happen)
import sklearn


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [2]:
#imputation libraries (read this, then the cell below, then this cell again)
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

AttributeError: module 'sklearn' has no attribute 'neighbors'

In [3]:
#imputation libraries
from missingpy import MissForest

ModuleNotFoundError: No module named 'missingpy'

### Manipulate dataframe

Data can be downloaded for the plasma proteomic dataset from the following paper: <br>
Multi-Omics Resolves a Sharp Disease-State Shift between Mild and Moderate COVID-19 (Su et al., 2020)

In [ ]:
data = pd.read_excel('../Data/Su_raw.xlsx', sheet_name=3)   #Sheet name = 3, proteomic     (sheet name = 4, metabolomic)

controls = data.loc[data['Group'] == 'Healthy Donor ']
cases = data.loc[data['Group'] == 'COVID19 ']

#There are two readings for each patient e.g. INCOV051-BL and INCOV051-AC
#BL stands for the first draw (this is the one we want); AC is the second draw and can be filtered out
first_draws = cases[cases.loc[:,'sample_id'].str.contains("BL")]
first_draws.loc[:,'sample_id'] = first_draws.loc[:,'sample_id'].str.rstrip('-BL')  #remove 'BL' label from the cases (so I can match to WHO status)

df = pd.concat([controls, first_draws], ignore_index=True)  #concatenate dataset and reset index 
df.insert(0, "WHO_status", '0') #insert new column for WHO_status and temporarily add all zeros

#display(df['sample_id'][124:130])    #first instance of a case is at index 124
display(df.iloc[:,3:])


In [ ]:
#Read in WHO status data
who_data = pd.read_excel('../Data/Su_raw.xlsx', sheet_name=1)

#Subset to only first draw readings
who_data = who_data[who_data["Sample ID"].str.contains("-1")]
who_data = who_data.reset_index(drop=True) #Reset index and drop old index
display(who_data)


#Create a dictionary with the sample names as a key and WHO status as a value
who_dict = {}

for i in range(0,len(who_data.index)):
    who = who_data["Who Ordinal Scale"][i]
    if who == 1 or who == 2 or who == "1 or 2":
        who = '1-2'
    elif who == 3 or who == 4:
        who = '3-4'
    elif who == 5 or who == 6 or who == 7:
        who = '5-7'
    else:
        print(who)
        who = "ERROR"

    sample = who_data["Study Subject ID"][i]
    who_dict[sample] = who



print(who_dict)

In [ ]:
#Add WHO data into the original dataframe
for i in range(124,len(df.index)): #first instance of a case is at index 124
    sample = df.loc[i,"sample_id"]
    if sample in who_dict:
        df.loc[i,"WHO_status"] = who_dict[sample]

display(df)

In [ ]:
#Make a different dataframe containing the meta data only (this will be changed later when samples are removed)
df.set_index('sample_id', inplace=True)
df.columns.values.tolist()

meta_data = df.iloc[:,[0,1]]
display(meta_data)

In [ ]:
df_num  = df.iloc[:,2:] #all rows, all columns apart from the first three which is meta data
df_num = df_num.set_index('sample_id')

#Percentage of missing data in dataframe
missing_vals = df_num.isna().sum().sum()
print((missing_vals/(259*464))*100) #1.6% missing data for proteomic
#print((missing_vals/(266*1050))*100) #19.3% for metabolomic

In [ ]:
df_num

In [ ]:
#Identify which columns have missing data
#print(df_num.isna().any()[lambda x: x])

#Remove features with >40% missing data - same as Sara's thesis
df_filter = df_num.loc[:, df_num.isin([' ', np.nan, 0]).mean() <= 0.4]    #keep all rows, if columns have ' ' or nan or 0, count as though missing and take as TRUE, get the mean of all the TRUE values
print(df_filter.isna().any()[lambda x: x]) #Identify which columns still have missing data


#For metabolomic 895 metabolites left after filtering


In [ ]:

#Identify which rows have missing data
#print(df_filter[df_filter.isna().any(axis=1)])

#identify which ROWS have missing values for each sample
df_missing_row = df_filter[df_filter.isna().any(axis=1)]
for index, row in df_missing_row.iterrows():
    for column in df_missing_row.columns:
        if pd.isnull(row[column]):
            print(index,column)

#Remove samples with >50% missing data - same as Sara's thesis (none of the samples were removed)
df_filter = df_filter.loc[df_filter.isin([' ', np.nan, 0]).mean(axis=1) <= 0.5,:]    

missing_vals = df_filter.isna().sum().sum()    #.sum() takes sum of missing values in each column, so .sum() again to take sum of all column values
print((missing_vals/(259*454))*100)  #259 rows, 454 proteins

display(df_filter)

In [ ]:
#Impute missing data (ignore the warning messages)
imputer = MissForest()
X_imputed = imputer.fit_transform(df_filter.to_numpy())

In [ ]:
#Check imputed data to make sure there are no missing values
df_impute = pd.DataFrame(X_imputed,columns=df_filter.columns, index=df_filter.index)
missing_vals = df_impute.isna().sum().sum()
print((missing_vals/(259*464))*100) #no missing values


In [ ]:
#The data has already gone through log2normalisation (negative values present frmo values < 1)
#Taking log2 of a number under 1 will give a negative value
#Note: log2 transforming is a normalisation step, which usually reduces the skew in the data, so that the data can be scaled using StandardScaler()
print(df_impute.max().max())
print(df_impute.min().min())
print(df_impute.mean(axis = 0)) #if scaled -> mean of 0   (not scaled yet)
print(df_impute.std(axis = 0)) #if scaled -> sd of 1       (not scaled yet)

display(df_impute)

In [ ]:
df_hist = df_impute.mean(axis = 0)
sns.histplot(df_hist, bins = 30,color='#BABAE2',edgecolor="k") 

#The mean value for each metabolite has been plotted
plt.title('Mean protein distribution before scaling',fontsize=16)
plt.xlabel('Protein expression',fontsize=13)
plt.ylabel('Count',fontsize=13) 

#plt.savefig( 'Figures/prescale_mean_protein_distribution.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:
df_impute_np = df_impute.to_numpy()
df_hist = df_impute_np.flatten()
sns.histplot(df_hist, bins = 30,color='#BABAE2',edgecolor="k") 

#The mean value for each metabolite has been plotted
plt.title('Protein distribution before scaling',fontsize=16)
plt.xlabel('Protein expression',fontsize=13)
plt.ylabel('Count',fontsize=13) 

#plt.savefig( 'Figures/prescale_protein_distribution.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:
#I decided not to use because too stringent (>20 outliers detected)

# Carry out PCA to identify outliers  
#https://towardsdatascience.com/outlier-detection-using-principal-component-analysis-and-hotellings-t2-and-spe-dmodx-methods-625b3c90897

#from pca import pca

# Initialize pca to also detected outliers
#model = pca(normalize=True, detect_outliers=['ht2', 'spe'], n_std=2  )

# Fit and transform
#results = model.fit_transform(df_impute)

# Get the outliers using Hotellings T2 method
#df_impute.loc[results['outliers']['y_bool'], :]

# Get the outliers using SPE/DmodX method
#df_impute.loc[results['outliers']['y_bool_spe'], :]

In [ ]:
#Conduct PCA to identify outliers

features = df_impute.columns[:]
x = df_impute.loc[:, features].values

pca = PCA(n_components=20) 
principal_components = pca.fit_transform(x)


In [ ]:
#Use elbow method for scree plot to determine number of PCs to use to calculate residuals (4 principal components)
print(sum(pca.explained_variance_ratio_))
PC_values = np.arange(pca.n_components_) + 1
plt.plot(PC_values, pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')


sns.set_style("white")
sns.lineplot(x=PC_values, y=pca.explained_variance_ratio_,  color='#42A7A9',marker="o")   # marker="o" shows markers as dots
plt.title('Scree Plot',fontsize=20)
plt.xlabel('Principal Component Number',fontsize=15)
plt.ylabel('Variance Explained',fontsize=15)

#plt.savefig( 'Figures/Scree_plot_protein_QC.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')


### Individual residual variance method to identify outliers

In [ ]:
#Conduct PCA to identify outliers

features = df_impute.columns[:]
x = df_impute.loc[:, features].values

pca = PCA(n_components=4) #determined using scree plot in the above cell
principal_components = pca.fit_transform(x)



#Calculate residuals to identify outliers
#Formula is:     Residuals = Data matrix - (PCA scores * PCA loadings)  https://datascience.stackexchange.com/questions/29132/how-can-i-extract-the-residual-array-from-the-scikit-learn-pca-routine

#Obtain PCA loadings (https://stackoverflow.com/questions/21217710/factor-loadings-using-sklearn)
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

residuals = df_impute - principal_components.dot(loadings.transpose())
display(residuals)

#Calculate individual residual variance by taking the Euclidean norm (L2 norm) of values for each sample
#https://stackoverflow.com/questions/7741878/how-to-apply-numpy-linalg-norm-to-each-row-of-a-matrix
l2_norm = pd.DataFrame(np.linalg.norm(residuals,axis=1),columns = ['Individual residual variance'], index = df.index)
l2_norm_unordered = l2_norm.copy()
l2_norm = l2_norm.sort_values(['Individual residual variance'], ascending=[False])
display(l2_norm[:10])

#Top 10 outliers
outliers = list(l2_norm[:10].index.values)



In [ ]:
#Preparing data for bar graph 
l2_norm_unordered['Sample ID'] = l2_norm_unordered.index
l2_norm_unordered = pd.concat([l2_norm_unordered, meta_data], axis = 1)
l2_mean = l2_norm_unordered['Individual residual variance'].mean()
l2_std = l2_norm_unordered['Individual residual variance'].std()


#Bar graph showing individual residual variance with mean/median line
g = sns.barplot(data=l2_norm_unordered,x="Sample ID", y="Individual residual variance",hue='Group',width=2)

g2 = g.axes
g2.set(xticklabels=[],xticks=[]) 
g2.axhline(l2_mean,color='r') #plot mean
g2.axhline(l2_mean +(l2_std*3),color='r') #plot 3 standard deviations away from the mean
g2.margins(x=0.01) #increase space between first and last bar and the edges 

#Add labels for outliers
g2.text(3,610,"1027114")
g2.text(36,510,"1347882")
g2.text(215,430,"Mean + 3sd")
g2.text(240,190,"Mean")  

#plt.savefig( 'Figures/individual_residual_variance.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')


In [ ]:
#df_hist = df_impute.mean(axis = 0)
sns.histplot(l2_norm, bins = 40,color='#BABAE2',edgecolor="k",legend=False) 

plt.xlabel('Individual residual variance',fontsize=13)
plt.ylabel('Count',fontsize=13) 

#plt.savefig( 'Figures/individual_residual_variance_distribution.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')


### Counting number of outliers outside of mean plus/minus 3 standard deviations

In [ ]:
#Outlier detection step - Z-score method
#The outliers should be removed before scaling
df_norm = pd.DataFrame(StandardScaler().fit_transform(df_impute),columns=df_impute.columns, index=df_impute.index)

df_norm = df_norm.to_numpy()
outlier_count = np.sum((df_norm > 3) | (df_norm < -3),axis=1)
outlier_df = pd.DataFrame(outlier_count, index = meta_data.index, columns = ['Outlier number'])
outlier_df = outlier_df.sort_values(['Outlier number'], ascending=[False])



#I originally determined the outliers by taking the outliers that are 3 standard deviations above the mean,
#however if you look at the histogram below this is not accurate because the distribution is not a NORMAL DISTRIBUTION!
#therefore it's better to just plot a histogram or barplot and determine by eye
display(outlier_df[:10]) #Outliers 3 std above the mean (first 3)

outlier_df['Outlier number'].mean()  #3.8108
outlier_df['Outlier number'].std()  #12.17


In [ ]:
#df_hist = df_impute.mean(axis = 0)
sns.histplot(outlier_df, bins = 50,color='#BABAE2',edgecolor="k",legend=False) 

plt.title('Outlier count',fontsize=16)
plt.xlabel('Number of outliers',fontsize=13)
plt.ylabel('Count',fontsize=13) 

#plt.savefig( 'Figures/outlier_histogram.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')


### Identifying outliers using pair plot method

In [ ]:
#Pair plots for first 4 principal components
df2 = pd.DataFrame(data = principal_components, columns = ['PC1', 'PC2','PC3', 'PC4'], index = df.index)

#Concatenate WHO information
df3 = pd.concat([df2, meta_data], axis = 1)
#display(df3)

sns.pairplot(df3, hue='Group',height=1.8)   

#plt.savefig( 'Figures/outlier_pairplot.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')


In [ ]:
#Alternatively, can dentify outliers by looking at which points are at the extremes of the PC axes and checking with the PCA plot
outliers_pc = df3.sort_values(['PC4'], ascending=[False])
display(outliers_pc.iloc[:10,:])

### Visualising outliers with first 2 principal components

In [ ]:
#PCA for visualisation WITH FIRST 2 PRINCIPAL COMPONENTS AND OUTLIERS LABELLED
features = df_impute.columns[:]
x = df_impute.loc[:, features].values

pca = PCA(n_components=2) #two principal components for data visualisation
principal_components = pca.fit_transform(x)

df2 = pd.DataFrame(data = principal_components, columns = ['PC1', 'PC2'], index = df.index)

#Concatenate WHO information
df3 = pd.concat([df2, meta_data], axis = 1)

display(df3)

#Subset outlier dataframe to plot outliers
outlier_df = df3[df3.index.isin(outliers)]



In [ ]:
outlier_df

In [ ]:

sns.set_style("white")

sns.lmplot(
    x='PC1', 
    y='PC2', 
    data=df3, 
    hue='WHO_status', 
    hue_order = ['0', '1-2', '3-4','5-7'],
    fit_reg=False, 
    legend=False,
    scatter_kws={"s": 20}
    )
#Note: I don't use the seaborn legend but check it matches with the seaborn legend

for index,outlier in (enumerate(outliers)):
    plt.text(df3.PC1[outlier],df3.PC2[outlier], outlier)


plt.title('PCA for outlier detection',fontsize=20)
plt.xlabel('PC1 (' + str(round(pca.explained_variance_ratio_[0]*100,2)) + '%)',fontsize=15)
plt.ylabel('PC2 (' + str(round(pca.explained_variance_ratio_[1]*100,2)) + '%)',fontsize=15)
plt.legend(framealpha=1, frameon = 'True', title="WHO status",title_fontsize='large', prop={'size': 14}, bbox_to_anchor=(1.04, 0.7)) 
#This has more information on the bbox_to_anchor coordinates: https://stackoverflow.com/questions/4700614/how-to-put-the-legend-outside-the-plot

#plt.savefig( 'Figures/PCA_protein_QC.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')


### Removal of outliers prior to scaling

In [ ]:
#Remove outlier 1027114
df_impute2 = df_impute.drop(1027114)
meta_data = meta_data.drop(1027114)


In [ ]:
#Normalise by column i.e. each pathway is normalised across all patients
#mean of the observed values becomes 0 and the standard deviation is 1


df_norm = pd.DataFrame(StandardScaler().fit_transform(df_impute2),columns=df_impute2.columns, index=df_impute2.index)

print(df_norm.max().max())
print(df_norm.min().min())
print(df_norm.mean(axis = 0)) #mean of 0  
print(df_norm.std(axis = 0)) #sd of 1


In [ ]:
#One feature has a lower mean than the others (superoxide dismutase 2, mitochondrial)
mean = pd.DataFrame(df_norm.mean(axis = 0))
mean[[0]].idxmin()

In [ ]:
df_hist = df_norm.mean(axis = 0)
sns.histplot(df_hist, bins = 50,color='#79C99E',edgecolor="k") 

#The mean value for each metabolite has been plotted
plt.title('Mean protein distribution',fontsize=16)
plt.xlabel('Protein expression (e-15)',fontsize=13)
plt.ylabel('Count',fontsize=13) ;

#plt.savefig( 'Figures/mean_protein_distribution.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:

df_norm_np = df_norm.to_numpy()

df_hist = df_norm_np.flatten()
sns.histplot(df_hist, bins = 45,color='#79C99E',edgecolor="k") 

#The mean value for each metabolite has been plotted
plt.title('Protein distribution',fontsize=16)
plt.xlabel('Protein expression',fontsize=13)
plt.ylabel('Count',fontsize=13) ;

#plt.savefig( 'Figures/protein_distribution.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:
#Carry out PCA

features = df_norm.columns[:]
x = df_norm.loc[:, features].values

pca = PCA(n_components=2)
principal_components = pca.fit_transform(x)
df2 = pd.DataFrame(data = principal_components, columns = ['PC1', 'PC2'])

#Restore original index
df2 = df2.set_index(df_norm.index)

#Concatenate WHO information
df3 = pd.concat([df2, meta_data], axis = 1)

#display(df3)




sns.set_style("white")


sns.lmplot(
    x='PC1', 
    y='PC2', 
    data=df3, 
    hue='WHO_status', 
    hue_order = ['0', '1-2', '3-4','5-7'],
    fit_reg=False, #don't draw line of best fit
    legend=False,
    scatter_kws={"s": 20}
    )
#Note: I don't use the seaborn legend but check it matches with the seaborn legend


plt.title('PCA for proteins',fontsize=20)
plt.xlabel('PC1 (' + str(round(pca.explained_variance_ratio_[0]*100,2)) + '%)',fontsize=15,)
plt.ylabel('PC2 (' + str(round(pca.explained_variance_ratio_[1]*100,2)) + '%)',fontsize=15)
plt.legend(framealpha=1, frameon = 'True', title="WHO status",title_fontsize='large', prop={'size': 14}, bbox_to_anchor=(1.04, 0.7)) 

#plt.savefig( 'Figures/proteins_PCA.png' , dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')


       

In [ ]:
#Add metadata to the end of the df
df_final = pd.concat([df_norm, meta_data],axis=1) 
#df_final.to_csv('Data/Su_COVID_proteomics_processed_names.csv')

### Convert protein columns from names to UniProt IDs

In [ ]:
#Convert protein columns from names to Uniprot IDs 

#Read in pre-processed dataframe
df = pd.read_csv('Data/Su_COVID_proteomics_processed_names.csv', index_col=0)

#Read in the conversion table
conversion_tab = pd.read_csv('Data/proteomics_metadata.csv', index_col=0)


In [ ]:
#Separate dataframe into numerical data and meta data
meta_data = df.iloc[:,-2:]
df = df.iloc[:,:-2]

In [ ]:
#Subset conversion table to only the proteins present in the data
protein_names = list(df.columns)
conversion_tab2 = conversion_tab[conversion_tab['gene_description'].isin(protein_names)]


In [ ]:
conversion_tab2 = conversion_tab2.sort_values(by='gene_description')

#Check duplicates in the gene description column have the same Uniprot IDs (they do)
conversion_tab2 = conversion_tab2.drop_duplicates(subset='uniprot')
conversion_tab2 = conversion_tab2.reset_index(drop=True)

from collections import Counter
mylist = conversion_tab2.gene_description

dups = [k for k,v in Counter(mylist).items() if v>1]
print(dups)



In [ ]:
df_uniprot = df.rename(columns=dict(zip(conversion_tab2["gene_description"], conversion_tab2["uniprot"])))

In [ ]:
#Add metadata to the end of the df
df_final = pd.concat([df_uniprot, meta_data],axis=1) 
#df_final.to_csv('Data/Su_COVID_proteomics_processed.csv')